# BS Model
- d1, d2 
- $\Delta$
- $\nu$
- Call & Put Price
- Implied Vol

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
def bs_call(S_o, K, r, T, sigma):
    d1 = (np.log(S_o/K) + ((r+(sigma**2)*0.5)*T))/(sigma*np.sqrt(T))
    d2 = d1 - (sigma*np.sqrt(T))
    c = (S_o*norm.cdf(d1)) - (K*np.exp(-1*r*T)*norm.cdf(d2))
    return c

def bs_put(S_o, K, r, T, sigma):
    d1 = (np.log(S_o/K) + ((r+(sigma**2)*0.5)*T))/(sigma*np.sqrt(T))
    d2 = d1 - (sigma*np.sqrt(T))
    p = -(S_o*norm.cdf(-d1)) + (K*np.exp(-1*r*T)*norm.cdf(-d2))
    return p

def bs_vega(S_o, K, r, T, sigma):
    d1 = (np.log(S_o/K) + ((r+(sigma**2)*0.5)*T))/(sigma*np.sqrt(T))
    vega = S_o * norm.pdf(d1) * np.sqrt(T)
    return vega

def bs_delta(S_o, K, r, T, sigma):
    d1 = (np.log(S_o/K) + ((r+(sigma**2)*0.5)*T))/(sigma*np.sqrt(T))
    delta = norm.cdf(d1)
    return delta

def get_implied_vol(S_o, K, r, T, c, initial_sigma = 0.1, threshold = 0.0001,max_iterations=1000):
    sigma = initial_sigma
    for i in range(max_iterations):
        calculated_call = bs_call(S_o, K, r, T, sigma)
#         print(calculated_call)
        vega = bs_vega(S_o, K, r, T, sigma)
#         print(vega)
        diff = c - calculated_call
        if abs(diff)<threshold:
            return sigma
        sigma=sigma+diff/vega
#     print("maximum iterations used")
    return sigma


def BMS_d1(S, K, r, q, sigma, tau):
    ''' Computes d1 for the Black Merton Scholes formula '''
    d1 = 1.0*(np.log(1.0 * S/K) + (r - q + sigma**2/2) * tau) / (sigma * np.sqrt(tau))
    return d1

def BMS_d2(S, K, r, q, sigma, tau):
    ''' Computes d2 for the Black Merton Scholes formula '''
    d2 = 1.0*(np.log(1.0 * S/K) + (r - q - sigma**2/2) * tau) / (sigma * np.sqrt(tau))
    return d2

def BMS_price(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the Black Merton Scholes price for a 'call' or 'put' option '''
    tau = T - t
    d1 = BMS_d1(S, K, r, q, sigma, tau)
    d2 = BMS_d2(S, K, r, q, sigma, tau)
    if type_option == 'call':
        price = S * np.exp(-q * tau) * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    elif type_option == 'put':
        price = K * np.exp(-r * tau) * norm.cdf(-d2) - S * np.exp(-q * tau) * norm.cdf(-d1) 
    return price

def BMS_delta(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the delta for a call or a put. '''
    tau = T - t
    d1 = BMS_d1(S, K, r, q, sigma, tau)
    if type_option == 'call':
        delta = np.exp(-q * tau) * norm.cdf(d1)
    elif type_option == 'put':
        delta = np.exp(-q * tau) * (norm.cdf(d1) - 1)
    return delta

# Call Price = $\Delta$ Hedging

Cost of delta hedging in Black-Merton-Scholes (BMS) Model (Call) when r,q>0